In [14]:
import statistics as st

In [15]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from unet import UNet
from utils.data_vis import plot_img_and_mask
from utils.dataset import BasicDataset
import cv2

mask_threshold == out_threshold - закоменчено, не используется


In [16]:
def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(full_img))


    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        full_output = net(img)
        output = full_output[0]
        
        print('full_output.shape: ...')
        print(len(full_output))
        
        print('output.shape: ...')
        print(output.shape)
        
        probs_0 = torch.sigmoid(output[:, 0, :, :])
        probs_1 = torch.sigmoid(output[:, 1, :, :])
        probs_2 = torch.sigmoid(output[:, 2, :, :])
        probs_3 = torch.sigmoid(output[:, 3, :, :])
        probs_4 = torch.sigmoid(output[:, 4, :, :])
        probs_5 = torch.sigmoid(output[:, 5, :, :])

        probs_0 = probs_0.squeeze(0)
        probs_1 = probs_1.squeeze(0)
        probs_2 = probs_2.squeeze(0)
        probs_3 = probs_3.squeeze(0)
        probs_4 = probs_4.squeeze(0)
        probs_5 = probs_5.squeeze(0)

        tf = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize((full_img.height, full_img.width)),
                transforms.ToTensor()
            ]
        )

        probs_0 = tf(probs_0.cpu())
        probs_1 = tf(probs_1.cpu())
        probs_2 = tf(probs_2.cpu())
        probs_3 = tf(probs_3.cpu())
        probs_4 = tf(probs_4.cpu())
        probs_5 = tf(probs_5.cpu())
        
        mask_0 = probs_0.squeeze().cpu().numpy()
        mask_1 = probs_1.squeeze().cpu().numpy()
        mask_2 = probs_2.squeeze().cpu().numpy()
        mask_3 = probs_3.squeeze().cpu().numpy()
        mask_4 = probs_4.squeeze().cpu().numpy()
        mask_5 = probs_5.squeeze().cpu().numpy()
        
        full_mask = np.array([mask_0, mask_1, mask_2, mask_3, mask_4, mask_5])

    return full_mask

def get_output_filenames(args):
    in_files = args.input
    out_files = []

    if not args.output:
        for f in in_files:
            pathsplit = os.path.splitext(f)
            out_files.append("{}_OUT{}".format(pathsplit[0], pathsplit[1]))
    elif len(in_files) != len(args.output):
        logging.error("Input files and output files are not of the same length")
        raise SystemExit()
    else:
        out_files = output

    return out_files

def mask_to_image(mask):
    return Image.fromarray((mask * 255).astype(np.uint8))

In [17]:
model = 'ckpts_dir/axial_ckpts/best_epoch_18.pth'

net = UNet(n_channels=1, n_classes=6)

logging.info("Loading model {}".format(model))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
net.to(device=device)
net.load_state_dict(torch.load(model, map_location=device))

logging.info("Model loaded !")

In [18]:
scale = 1
mask_threshold = 0.5 #минимальная вероятность для рассматривания пикселя на маску

In [160]:
list_of_dirs = [
    '/home/natasha/unet/axial_data_new/test/First_3/', 
    '/home/natasha/unet/axial_data_new/test/Second_1/'
]

In [161]:
# img_list = os.listdir(list_of_dirs[0]+'pictures/') 
# masks_list = os.listdir(list_of_dirs[0]+'masks/') 

# img_src_pathes = []
# for img in img_list:
#     img_src_pathes.append(list_of_dirs[0]+'pictures/'+img)
    
# mask_src_pathes = []
# for mask in masks_list:
#     mask_src_pathes.append(list_of_dirs[0]+'masks/'+mask)

In [162]:
for i in range(0, len(list_of_dirs)):
    img_list = os.listdir(list_of_dirs[i]+'pictures/') 
    masks_list = os.listdir(list_of_dirs[i]+'masks/') 

    img_src_pathes = []
    for img in img_list:
        img_src_pathes.append(list_of_dirs[i]+'pictures/'+img)

    mask_src_pathes = []
    for mask in masks_list:
        mask_src_pathes.append(list_of_dirs[i]+'masks/'+mask)

## Test code

In [201]:
imgname = img_src_pathes[5]
maskname = mask_src_pathes[5]

img = Image.open(imgname)
true_mask = np.load(maskname)

mask = predict_img(net=net,
                   full_img=img,
                   scale_factor=scale,
                   out_threshold=mask_threshold,
                   device=device)

outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])


In [202]:
mask.shape

(6, 308, 320)

In [203]:
np.unique(mask[0, :, :])[:5]

array([0.        , 0.00392157, 0.00784314, 0.01176471, 0.01568628],
      dtype=float32)

In [204]:
true_mask.shape

(308, 320, 6)

In [205]:
res_treshold = 0.1
mask = np.array([mask[0] > res_treshold, mask[1] > res_treshold, mask[2] > res_treshold, \
                     mask[3] > res_treshold, mask[4] > res_treshold, mask[5] > res_treshold])

In [206]:
np.unique(mask[0, :, :])

array([False,  True])

In [207]:
d, w, h = mask.shape

In [208]:
true_mask2 = cv2.resize(true_mask[:,:,0], (h, w)) #тут берется только один канал граундтру маски, а у нас их шесть
# то есть либо в цикле делать, либо использовать какую-то другую операцию
true_mask2.shape

(308, 320)

In [209]:
pixelThreshold = 0.5
bin_mask = np.where(mask[0] > pixelThreshold, 1, 0) # опять берем только один канал, но уже предсказанной маски
bin_mask.shape

(308, 320)

In [223]:
# которые предсказались единицами и на месте которых в граунде стоят тоже единицы (если бинарно умножить, на их месте будут 1)
inter = bin_mask*true_mask2
unique_inter, counts_inter = np.unique(inter, return_counts=True)
true_pos = counts_inter[1]
# counts_inter, unique_inter

In [225]:
# которые предсказались единицами и на месте которых в граунде стоят нули
# в то же время реализована следующая мысль:

unique_mask, counts_mask = np.unique(bin_mask, return_counts=True)
# counts_mask[1]
false_pos = counts_mask[1] - true_pos

In [239]:
# которые предсказали нулями и на месте которых в граунде стоят единицы (если бинарно отнять, то на их месте будут -1)
diff = bin_mask - true_mask2
unique_diff, counts_diff = np.unique(diff, return_counts=True)
false_neg = counts_diff[0]

In [241]:
true_pos, false_pos, false_neg 

(1984, 211, 244)

In [242]:
precision = true_pos / (true_pos + false_pos)
recall = true_pos / (true_pos + false_neg)

In [243]:
precision

0.903872437357631

In [244]:
recall

0.8904847396768402

In [246]:
f = 2*(precision * recall) / (precision + recall)

In [247]:
f

0.8971286457155776

## Production code

In [277]:
f1_general = []
f1_channels = [[], [], [], [], [], [], []]

for i in range(0, len(img_src_pathes)):
    imgname = img_src_pathes[i]
    maskname = mask_src_pathes[i]

    img = Image.open(imgname)
    true_mask = np.load(maskname)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=scale,
                       out_threshold=mask_threshold,
                       device=device)
    
    res_treshold = 0.1
    mask = np.array([mask[0] > res_treshold, mask[1] > res_treshold, mask[2] > res_treshold, \
                     mask[3] > res_treshold, mask[4] > res_treshold, mask[5] > res_treshold])
    
    d, w, h = mask.shape
    
    
    for j in range(0, d):
        true_mask2 = cv2.resize(true_mask[:,:,j], (h, w))
        
        pixelThreshold = 0.5
        bin_mask = np.where(mask[j] > pixelThreshold, 1, 0)
        
        try:
        
            inter = bin_mask*true_mask2
            unique_inter, counts_inter = np.unique(inter, return_counts=True)
            true_pos = counts_inter[1]
            
            unique_mask, counts_mask = np.unique(bin_mask, return_counts=True)
            false_pos = counts_mask[1] - true_pos
            
            diff = bin_mask - true_mask2
            unique_diff, counts_diff = np.unique(diff, return_counts=True)
            false_neg = counts_diff[0]
            
            precision = true_pos / (true_pos + false_pos)
            recall = true_pos / (true_pos + false_neg)

            f = 2*(precision * recall) / (precision + recall)
            f1_general.append(f)
            f1_channels[j].append(f)
            
        except:
            
            f = 0
            print('hm')
    

outputs:
torch.Size([1, 6, 296, 320])
torch.Size([1, 64, 296, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 296, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
hm
hm
hm
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600

hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
t

hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
t

hm
hm
hm
hm
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 

hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
t

hm
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 6

hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600, 600])
full_output.shape: ...
10
output.shape: ...
torch.Size([1, 6, 308, 320])
hm
hm
hm
hm
outputs:
torch.Size([1, 6, 308, 320])
torch.Size([1, 64, 308, 320])
torch.Size([1, 6, 600, 600])
torch.Size([1, 6, 600

In [267]:
st.mean(f1_general)

0.47932987763919993

In [268]:
max(f1_general)

0.9557559236737262

In [269]:
min(f1_general)

0.0011958085895539025

In [271]:
st.mean(f1_channels[0])

0.7276366580928061

In [272]:
st.mean(f1_channels[1])

0.4939914723119183

In [273]:
st.mean(f1_channels[2])

0.20508178609923025

In [274]:
st.mean(f1_channels[3])

0.388078961691477

In [275]:
st.mean(f1_channels[4])

0.49959576896748115

In [276]:
st.mean(f1_channels[5])

0.2

In [189]:
with open('logs_score/log_'+model.split('/')[-1]+'_' + str(res_treshold) + 'my'+'.txt', 'w') as file:
    file.write('General metrics: \n')
    file.write('------------------------\n')
    file.write('Max dice loss:\n')
    file.write(str(max(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('Min dice loss:\n')
    file.write(str(min(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('Mean dice loss:\n')
    file.write(str(st.mean(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('\n')
    file.write('\n')
    
#     file.write('First metrics: \n')
#     file.write('------------------------\n')
    
#     file.write('Max f1:\n')
#     file.write(str(max(f1_first))+'\n')
#     file.write(' ---\n')

#     file.write('Min f1:\n')
#     file.write(str(min(f1_first))+'\n')
#     file.write(' ---\n')
    
#     file.write('Mean f1:\n')
#     file.write(str(st.mean(f1_first))+'\n')
#     file.write(' ---\n')
    
#     file.write('\n')
#     file.write('\n')